<a href="https://colab.research.google.com/github/vadigr123/colab-testing/blob/main/sender-gpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---

# 🚩 Start (Download files)

In [1]:
#@markdown ### ▶️ Download
#@markdown Use commas to download multiple files (URL,URL)
urls = "" # @param {type:"string"}

import os

# Create a folder for downloaded files
download_folder = "" # @param {type:"string"}
os.makedirs(download_folder, exist_ok=True)

# Split the string into individual URLs
url_list = urls.split(",")

# Download the files
for url in url_list:
    filename = url.split("/")[-1]  # Get the filename from the URL
    file_path = os.path.join(download_folder, filename)

    # Perform download using wget
    os.system(f'wget -O "{file_path}" "{url.strip()}"')

    print(f"✅ Downloaded: {filename}")

✅ Downloaded: manmosu_marimo.safetensors


---

In [ ]:
#@markdown Use if needed to send to HugginFace
!pip install huggingface_hub
from huggingface_hub import login, HfApi
import os

    # Token for Hugging Face login
token = ""  # @param {type:"string"}
repo_id = "" # @param {"type":"string"}
local_folder = "" # @param {"type":"string"}
remote_folder = "" # @param {"type":"string"}

    # Login
login(token=token)

    # Initialize the API
api = HfApi()

    # Iterate through all files in the folder
for filename in os.listdir(local_folder):
        file_path = os.path.join(local_folder, filename)  # Full file path
        remote_path = os.path.join(remote_folder, filename)  # Path on Hugging Face

api.upload_file(
    path_or_fileobj=file_path,
    path_in_repo=remote_path,
    repo_id=repo_id,
    repo_type="model",
    )
print(f"✅ Uploaded to Hugging Face: {filename}")

---

In [ ]:
#@markdown Use if needed to send to Github

!pip install PyGithub
from github import Github
import os

    # Token for GitHub login
github_token = ""  # @param {type:"string"}
repo_name = "" # @param {"type":"string"}
local_folder = "" # @param {"type":"string"}
remote_folder = "" # @param {"type":"string"}

    # Login
g = Github(github_token)
repo = g.get_repo(repo_name)

    # Iterate through all files in the folder
for filename in os.listdir(local_folder):
  file_path = os.path.join(local_folder, filename)
  remote_path = os.path.join(remote_folder, filename)  # Path on GitHub

with open(file_path, "rb") as file:
  content = file.read()

        # Upload the file to GitHub
repo.create_file(remote_path, f"Add {filename}", content)
print(f"✅ Uploaded to GitHub: {filename}")